# LAB 01: LAYOFFS 2023

## Mô tả đồ án:
- Từ nguồn dữ liệu lấy được từ Kaggle về vấn đề sa thải nhân viên ở các công ty trên toàn thế giới, đồ án này nhằm mục đích phân tích dữ liệu đã có để có được cái nhìn khách quan hơn về vấn đề này.

---

### Import các thư viện

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
from scipy import stats
from wordcloud import WordCloud
import calendar
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

---

### <span style="color:yellow"> A. Thu thập dữ liệu</span>


#### Ngữ cảnh thực hiện tìm kiếm dữ liệu

Sau dịch covid 19, các công ty công nghệ trên toàn cầu đang đối mặt với sự suy thoái kinh tế. Sự chậm lại của chi tiêu từ người tiêu dùng, lãi suất cao hơn từ các ngân hàng trung ương và sức mạnh của đồng đô la ngoại tệ đang gợi ý về khả năng xảy ra suy thoái kinh tế và các công ty công nghệ đã bắt đầu **cắt giảm nhân sự**.

#### Chủ đề

Phân tích tình trạng hỗn loạn công nghệ gần đây thông qua số liệu về việc sa thải nhân sự, từ đó rút ra được những thông tin hữu ích.

#### Nguồn
Dữ liệu được lấy trên Kaggle dưới file csv, đường link dẫn đến  [Ở đây](https://www.kaggle.com/datasets/swaptr/layoffs-2022?resource=download)

#### Người ta có cho phép sử dụng dữ liệu như thế này hay không?

Thông tin License của dữ liệu: Open Database.

Theo thông tin License của dữ liệu, người dùng được phép tự do chia sẻ, sửa đổi và sử dụng dữ liệu trong khi duy trì sự tự do tương tự cho người dùng khác.

#### Người ta đã thu thập dữ liệu này như thế nào? Phương pháp thực hiện là gì?

Dữ liệu được thu thập bằng cách theo dõi số liệu được báo cáo trên các nền tảng:
- Bloombreg
- San Francisco Business Times
- TechCrunch
- The New York Times

Phương pháp thu thập dữ liệu: Dữ liệu được tải xuống từ Layoffs.fyi Tracker và được biên soạn thành 1 file CSV.

#### Đọc dữ liệu

In [ ]:
df=pd.read_csv('./datasets/layoffs.csv')
df.head()

---

### <span style="color:yellow"> B. Khám phá dữ liệu</span>


#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

In [ ]:
# df['stage']

- Mỗi dòng chứa dữ liệu về một công ty và các thông tin liên quan đến công ty đó như: tên công ty, nơi đặt trụ sở chính, loại ngành nghề, số nhân viên bị sa thải, tỷ lệ sa thải, ngày công bố, giai đoạn của công ty, quốc gia công ty hoạt động, số vốn huy động bởi công ty.

- ví dụ: dòng 1: doanh nghiệp SiriusXM có trụ sở tại New York City và hoạt động trong ngành truyền thông, đã sa thải 475 nhân viên, chiếm 8% tổng số nhân viên của họ tính tới ngày 6 tháng 3 năm 2023. Công ty cũng đang ở giai đoạn Post-IPO và có trụ sở tại Hoa Kỳ. Họ đã huy động được số vốn 525 triệu đô la.

- Không có tình trạng các dòng có ý nghĩa khác nhau.

#### Mỗi cột có ý nghĩa gì?

In [ ]:

# print("total columns: ",len(df.columns)) 

# for column in range(len(df.columns)) :
#     print("column ", column," ", df.columns[column])
#     print("size: ", len(df.loc[:, df.columns[column]]))
#     print("number of values: ", len(df.loc[:, df.columns[column]].unique()), "\n")
#     print("head: ", ' ,'.join(df.loc[:, df.columns[column]].unique()) )

column=2
print("column ", column,": ", df.columns[column])
print("size: ", len(df.loc[:, df.columns[column]]))
print("number of values: ", len(df.loc[:, df.columns[column]].unique()))
# print(df.loc[:, df.columns[column]].unique())
arr= df.loc[:, df.columns[column]].unique()

nanIndex=-1
for index in range(len(arr)):
    if isinstance(arr[index], float) and np.isnan(arr[index]):
        nanIndex=index
        break
if nanIndex != -1:
    arr=np.delete(arr,nanIndex)

print("number of values (without nan): ", len(arr))

# print for string type
print("data (without nan) : ", ', '.join(arr))

# print for number type
# print( ", ".join(map(str, arr)))

    

## Thông tin của mỗi cột:
- Company: Tên của công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 1893
    - Một số Company có trong dữ liệu: Atlassian, SiriusXM, Alerzo, UpGrad, Loft, Embark Trucks,...
- Location: Nơi đặt trụ sở chính của công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 191
    - Một số Location có trong dữ liệu: Sydney, New York City, Ibadan, Mumbai, Sao Paulo,...

- Industry: Loại ngành mà công ty hoạt động
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 30
    - Một số Industry có trong dữ liệu: Other, Media, Retail, Education, Real Estate, Transportation,...

- Total_laid_off: Tổng số nhân viên bị sa thải bởi công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 286
    - Một số Total_laid_off có trong dữ liệu: 475.0, 400.0, 120.0, 340.0, 230.0, 100.0, 63.0, 30.0...

- Percentage_laid_off: Tỷ lệ nhân viên bị sa thải bởi công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 76
    - Một số Percentage_laid_off có trong dữ liệu: 0.05, 0.08, 0.15, 0.7, 0.03, 0.11, 0.1, 1.0...

- Date: Ngày công bố thông tin về việc sa thải
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 191
    - Một số Date có trong dữ liệu: 2023-03-06, 2023-03-03, 2023-03-02, 2023-03-01, 2023-02-28,...

- Stage: Giai đoạn của công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 17
    - Một số Stage có trong dữ liệu: Post-IPO, Series B, Unknown, Acquired, Series F, Series E...

- Country: Quốc gia nơi công ty đặt trụ sở
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 59
    - Một số Country có trong dữ liệu:  Australia, United States, Nigeria, India, Brazil,...

- Funds_raised: Tổng số vốn do công ty huy động (tính bằng triệu đô la)
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 639
    - Một số Funds_raised có trong dữ liệu: 525.0, 16.0, 631.0, 788.0, 317.0, 59.0, 152.0...

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [ ]:
df.dtypes

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    print(col,":",set(df[col].apply(type)))

Chúng ta nhận thấy rằng ba cột `industry`, `date` và `stage` có dạng dữ liệu không đồng nhất, tuy nhiên đây chỉ là do sự thiếu hụt dữ liệu trong các dòng, được biểu thị bằng giá trị `NaN` (dạng số thực).

#### Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

Các cột sau đang có kiểu dữ liệu là `object`, do đó cần được chuyển đổi sang kiểu chuỗi (`string`):

* `company`
* `location`
* `industry`
* `stage`
* `country`

Trong khi đó, cột `date` đang có kiểu dữ liệu là `object`, cần được chuyển đổi sang kiểu ngày tháng (`datetime`).

## Phân bố dữ liệu ở các cột

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào? 

In [ ]:
numerical_des = df.describe()
missing_ratio = pd.DataFrame({col : df[col].isnull().sum() / len(df) for col in numerical_des.columns}, index=["missing_ratio"])
pd.concat([numerical_des,missing_ratio])

Từ bảng dữ liệu phân tích của kiểu dữ liệu numberic, tồn tại những trường dữ liệu với tỉ lệ thiếu lên tới hơn 30%.

### Total Laid Off

In [ ]:
plt.figure(figsize=(14,6))
ax= sns.histplot(data=df, x='total_laid_off')
ax.set_yscale('log')
plt.show()

### Percentage Laid Off

In [ ]:
sns.histplot(data=df, x='percentage_laid_off')
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
ax=sns.histplot(data=df, x='funds_raised')
ax.set_yscale('log')
plt.show()

### Với mỗi cột không có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào? 

In [ ]:
cate_cols=['company', 'location', 'industry', 'stage', 'country']
cate_rows=["missing_ratio","num_diff_vals","diff_vals"]
data_cate={}
for col in cate_cols:
    row=[]
    row.append(round(df[col].isnull().sum()/len(df[col])*100,3))
    data_cate.update({f'{col}':row})

    row.append(len(df[col][df[col].isna()==False].unique()))
    row.append(np.array(df[col][df[col].isna()==False].unique()))
    data_cate.update({f'{col}':row})

cate_col_df=pd.DataFrame(data_cate,index=cate_rows)
display(cate_col_df)

#### Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

In [ ]:
df.info()

Quan sát thấy có số lượng lớn dữ liệu bị thiếu và có những cột phải xử lý kiểu dữ liệu do vậy ta phải tiền xử lý dữ liệu.

### Tiền xử lý dữ liệu

In [ ]:
# Kiểm tra missing data
have_Nan=df.columns[df.isna().any()]

# Đếm số missing data của các cột có missing data
print("Missing data:")
print(df[have_Nan].isnull().sum())
df.isna().sum().plot(kind="barh",title='Missing data')
plt.show()

# Phần trăm dữ liệu bị thiểu của những cột này
print("Percentage missing data:")
percentage_missing_data=df[have_Nan].isnull().sum() * 100 / df.shape[0]
print(percentage_missing_data)

# Kiểm tra có dữ liệu bị duplicate không
print("Number of rows duplicated: "+str(df[df.duplicated()].shape[0]))


Ta thấy có một số lượng khá lớn dữ liệu bị thiếu ở cột `total_laid_off`,`percentage_laid_off` và `funds_raised` . Vì có những cột dữ liệu bị thiếu lên đến hơn 30% nên ta không thể chọn phương án bỏ toàn bộ những dòng nào bị thiếu dữ liệu. Bên cạnh đó có tồn tại 1 dòng bị trùng ta cũng cần phải xử lý. 

- Đầu tiên ta loại bỏ những dòng có dữ liệu bị thiếu ở cột `industry`,`date` và `stage` vì số lượng bị thiếu không đáng kể.
- Xóa dòng bị trùng dữ liệu.
- Loại bỏ những dòng mà bị thiếu cả hai trường quan trọng là `total_laid_off` và `percentage_laid_off` vì những dòng này không có ý nghĩa để ta phân tích.
- Tiếp theo tùy theo hướng phân tích ta sẽ chọn những cột dữ liệu phù hợp. tạm thời ta sẽ fill bằng 0.

In [ ]:
# Loại bỏ những dòng có dữ liệu bị thiếu ở cột `industry`,`date` và `stage`
df = df[~df['industry'].isna()]
df = df[~df['date'].isna()]
df = df[~df['stage'].isna()]

# Xóa dòng bị trùng dữ liệu.
df = df[~df.duplicated()]

# Loại bỏ những dòng mà bị thiểu cả hai trường quan trọng là `total_laid_off` và `percentage_laid_off`
df = df[~(df['total_laid_off'].isna()&df['percentage_laid_off'].isna())]

# 
df = df.fillna(0)
print("After fill null:")
print(df.isna().sum())

Sau khi đã tạm thời xử lý các dữ liệu bị thiếu ta bắt đầu xử lý kiễu dữ liệu của các cột.

- Chuyển data type của cột `date` sang dạng `datetime`.
- Chuyển data type của các cột đang là dạng `object ('str')` sang dạng `category`

In [ ]:
# Chuyển data type của cột `date` sang dạng `datetime`.
df['date']=pd.to_datetime(df['date'])
# Chuyển data type của các cột đang là dạng `object` sang dạng `category`
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    df[col]=df[col].astype('category')
df.info()

Đã xữ lý xong kiểu dữ liệu của các cột. Giờ ta quan sát đến giá trị dữ liệu bên trong các cột.

In [ ]:
# Sắp xếp df theo tên company để quan sát cột `company`
print(df.sort_values('company')['company'].unique())
a=df['company'].unique()
b=df['company'].str.upper().unique()
print(a.shape[0])
print(b.shape[0])

Số lượng tên công ty:

-   Data gốc: 1629
-   Data sau khi đưa tất cả về uppercase: 1624

Ta thấy có sự khác nhau về số lượng tên công ty khi ta giữ nguyên data gốc và sau khi đưa tất cả về uppercase -> Có những tên của cùng một công ty nhưng chưa được chuẩn hóa cùng 1 quy tắc.
-> Đưa tất cả về 1 loại (uppercase) và ta cũng phải xóa các khoảng trắng thừa trong tên công ty phòng trường hợp có tên công ty bị thừa khoảng trắng.

In [ ]:
# Xử lý dữ liệu cột `company`
df['company']=df['company'].str.upper()
df['company']=df['company'].str.split().str.join(" ")
df['company'].unique().shape[0]

Xử lý tương tự cho cột `location`, `industry`, `stage` và `country`.

In [ ]:
# Xử lý dữ liệu cột `location`
df['location']=df['location'].str.upper()
df['location']=df['location'].str.split().str.join(" ")
df['location'].unique().shape[0]

In [ ]:
# Xử lý dữ liệu cột `industry`
df['industry']=df['industry'].str.upper()
df['industry']=df['industry'].str.split().str.join(" ")
df['industry'].unique().shape[0]

In [ ]:
# Xử lý dữ liệu cột `stage`
df['stage']=df['stage'].str.upper()
df['stage']=df['stage'].str.split().str.join(" ")
df['stage'].unique().shape[0]

In [ ]:
# Xử lý dữ liệu cột `country`
df['country']=df['country'].str.upper()
df['country']=df['country'].str.split().str.join(" ")
df['country'].unique().shape[0]

Tiếp theo ta kiểm tra những cột có kiễu dữ liệu là số để xem có xuất hiện dữ liệu không hợp lệ không.
- Có dòng nào chứa số âm không?
- `percentage_laid_off` có giá trị nào lớn hơn 1 không?

In [ ]:
# Kiểu tra giá trị âm
print("Negative total_laid_off: "+str(sum(df.total_laid_off<0)))
print("Negative percentage_laid_off: "+str(sum(df.percentage_laid_off<0)))
print("Negative funds_raised: "+str(sum(df.funds_raised<0)))

# Kiểm tra giá trị `percentage_laid_off` không hợp lệ
print("percentage_laid_off greater than 1: "+str(sum(df.percentage_laid_off>1)))

Sau khi đã xử lý xong tất cả ta sắp xếp và reset index của data.

In [ ]:
df=df.sort_values('company')
df=df.reset_index(drop=True)
df.head()

### Thêm cột total_employee để phục phụ phân tích sau này

In [ ]:


df_with_total_employees=df.sort_values(by='total_laid_off',ascending=False)
df_with_total_employees=df_with_total_employees.reset_index(drop=True)
df_with_total_employees=df_with_total_employees[df_with_total_employees['percentage_laid_off']>0]
df_with_total_employees['total_employees']=round(df_with_total_employees['total_laid_off']/df_with_total_employees['percentage_laid_off'])
df_with_total_employees=df_with_total_employees[df_with_total_employees['total_employees']>0]
df_with_total_employees=df_with_total_employees.sort_values(by='total_employees',ascending=False)
df_with_total_employees=df_with_total_employees.reset_index(drop=True)



In [ ]:
df2=df_with_total_employees.groupby('company')['total_employees'].max()
df2.sort_values(ascending=False).head(10)

### Tìm hiểu mối liên hệ giữa Stage và Percentage_laid_off: 

In [ ]:
df_clean_percentage=df[df['percentage_laid_off']>0]
df_clean_percentage['percentage_laid_off']=df_clean_percentage['percentage_laid_off']*100
df_clean_percentage['percentage_lo_range']=pd.cut(x=df_clean_percentage['percentage_laid_off'], bins=[0, 10, 20, 30,
                                          40,50,60,70,80,90,100],labels=['0-10', '10-20', '20-30',
                                          '30-40','40-50','50-60','60-70','70-80','80-90','90-100'],include_lowest=True)

In [ ]:
df_clean_percentage['stage']=df_clean_percentage['stage'].sort_values()
table = pd.pivot_table(df_clean_percentage, values='percentage_laid_off', index='percentage_lo_range',columns='stage', aggfunc='count')
table=table.sort_values(by=['percentage_lo_range'],ascending=False)


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15, 5), tight_layout=True)

df_clean_percentage=df_clean_percentage.sort_values('stage',ascending=True)
fig.tight_layout(pad=5.0)
ax1.set_xlabel('Percentage laid off (%)')
plt.xticks(rotation=45)
ax1.set_title('Phân bố giữa Stage, percentage laid off với mean',wrap=True)
ax1.set_xticklabels(df_clean_percentage['stage'].unique(),rotation=45)

sns.scatterplot(x='stage', y='percentage_laid_off',data=df_clean_percentage, ax=ax1,alpha=0.5)

sns.lineplot(x='stage', y='percentage_laid_off',data=df_clean_percentage,ax=ax1,color='red',ci=None)


sns.heatmap(table, annot=True, fmt="d", linewidths=.5, ax=ax2,cmap="crest",cbar_kws={'label': 'Number of companies'})
ax2.set_title('Stage, quy mô lay off và tổng số công ty diễn ra lay off với quy mô đó',wrap=True)
ax2.set_xlabel('Percentage laid off (%)')
plt.xticks(rotation=45)
plt.show()

Một vài điểm đáng chú ý:
- Nhìn vào biểu đồ scatter và heatmap ta có thể thấy dữ liệu phân bố chủ yếu ở mức sa thải dưới 20% nhân sự mỗi đợt. 
- Nhìn và heat map ta thấy tương đồng nhiều nhất là 150 công ty ở stage `Post-IPO` có xu hướng sa thải >10% nhân sự.
- Nhìn vào mean line, các công ty có `stage` là `Seed` sa thải % nhân sự rất cao và không bao giờ sa thải dưới 10% nhân sự.
- Dữ liệu chủ yếu tập trung ở nhóm các công ty có stage từ `Series A` đến `Series E`. Nhóm này có xu hướng sa thải trung bình 10-20% mỗi đợt.
- Các công ty stage từ `Series F` đến `Series J` và `Subdiary` có rất ít dữ liệu sa thải và không bao giờ sa thải hơn 50% nhân sự trong một đợt. Có thể lý giải đây là các giai đoạn gọi vốn sau khi đã khá ổn định của các công ty nên ít diễn ra sa thải. Các công ty thuộc `Subdiary` thì có công ty mẹ chống lưng nên tình trạng cũng tương đối ổn định.

### Stage nào có tổng số laid off cao nhất:

In [ ]:
stage_df = df.groupby('stage').sum().reset_index()

In [ ]:
px.pie(stage_df,names='stage',values='total_laid_off',
      color_discrete_sequence=px.colors.sequential.thermal,
       title='Tổng số laid off của từng stage (theo tỷ lệ %)')

Từ biểu đồ tổng số laid off của từng stage ta có thể thấy:
- Stage POST-IPO chiếm 53.2% tổng số lượng laid off, chiếm hơn 1/2 tổng số laid off của tất cả các công ty.
- POST-IPO cũng chính là Stage có tổng số laid off cao nhất.

### Tổng số lượng nhân viên bị sa thải của các ngành công nghiệp khác nhau theo từng năm: 

In [ ]:
df = df.reset_index()
df['date'] = pd.to_datetime(df['date'])
df_industry = df.groupby([ df.industry, df.date.dt.year]).sum()
# df_industry.sort_values(by=['total_laid_off','date'], ascending=False)

In [ ]:
df_industry = df_industry.reset_index()

In [ ]:
plt.figure(figsize=(14, 10))
plt.xticks(rotation=90)
plt.title("Tổng số lượng sa thải của các ngành công nghiệp khác nhau trong từng năm")
sns.set(style="white", palette="Accent", color_codes=True)

sns.barplot(data=df_industry.sort_values(by=['total_laid_off','date'], ascending=False), x="industry", y="total_laid_off", hue="date")

Xu hướng sa thải hằng năm của thế giới cho thấy rằng:
- Việc xa thải cao hơn vào năm 2020, sau đó giảm và năm 2021 và tăng vọt vào năm 2021.
- Vào năm 2020, các ngành công nghiệp Transportation (vận tải) và Travel (du lịch) đã bị ảnh hưởng nhiều nhất bởi dịch COVID-19. Điều đó được thể hiện thông qua số lượng sa thải vào năm 2020. Nhưng 2 ngành này đã phục hồi vào những năm sau đó.
- Các ngành công nghiệp Retail (bán lẻ), Consumer (tiêu dùng) và Food (thực phẩm) đã phải đối mặt với việc sa thải gia tăng mạnh mẽ qua các năm.

### Theo từng năm, quốc gia có tổng số lượng sa thải lớn nhất:

In [ ]:
df['year'] = df['date'].dt.year
layoff_year_country = df.groupby(['year','country']).sum().reset_index()[['year','country','total_laid_off']]

In [ ]:
layoff_year_country = layoff_year_country.sort_values(["year","total_laid_off"],ascending=False)

In [ ]:
px.bar(layoff_year_country,x='year',y='total_laid_off',color='country',text='country',
      title='Tổng lượng sa thải của từng năm theo quốc gia'
      )

Từ biểu đồ trên ta thấy:
- Tổng số lượng sa thải giảm trong giai đoạn từ 2020 sang 2021. Nhưng sau đó lại tăng mạnh vào giai đoạn 2022.
- Với tất cả các năm 2020, 2021, 2022, 2023 thì United States luôn là quốc gia có số lượng sa thải lớn nhất. Thường chiếm gần 1/2 tổng số lượng sa thải của tất cả các quốc gia.

### Mối quuan hệ giữa total layoffs và funds raised của các công ty:

In [ ]:
funds_raised = df[(df.funds_raised<40000)][['total_laid_off', 'funds_raised']]

In [ ]:
fig = plt.figure(figsize=(15,10))
sns.regplot(data=df, x=funds_raised.total_laid_off, y=funds_raised.funds_raised)
plt.title("Mối quan hệ giữa total layoffs và funds raised của các công ty")
plt.show()

Dựa vào biểu đồ Mối quan hệ của total laidoffs và funds raised:
- Đa số các công ty có số funds raised vừa và ít thì cũng có total laidoffs vừa và thấp.
- Những công ty có total laidoff cao, thậm chí là rất cao có funds raised rất thấp (gần bằng 0).
- Ngược lại, những công ty có funds raised cao thì thường sẽ có total laidoff thấp.

### Xét từng quốc gia, thể hiện bằng biểu đồ cột cho thấy số lượng người bị sa thải của các ngành. Trí 

### Số lượng sa thải trên thế giới giai đoạn 2020 - 2023

In [ ]:
# group the data by country and industry and sum the total_laid_off column
grouped = df.groupby(['country']).agg({'total_laid_off': 'sum'}).reset_index()
# create a choropleth map using Plotly
fig = px.choropleth(grouped, locations='country', locationmode='country names', color='total_laid_off',
                    # color_continuous_scale='OrRd',
                    color_continuous_scale='YlOrRd',
                    hover_name='country', projection='natural earth', title='Number of People Laid Off by Country')

# show the map
fig.show()

Biểu đồ trên thể hiện tình hình người bị sa thải cụ thể ở các đất nước trên thế giới. Cụ thể:
- Đất nước đang đứng dưới nguy cơ lớn về việc chuyển đổi nhân sự là Mỹ với thống kê khoảng 250.000 người mất việc.
- Các đất nước khác thì có tỉ lệ ít đáng kể so với Mỹ, như với Ấn Độ với khoảng 35.953 người mất việc, Brazil 10.391 người, Canada 6.319 người, các đất nước còn lại với con số không đáng kể.

*Vì số lượng ngành nghề khá nhiều để visualize, nên ta sẽ nhóm các ngành nghề thành các nhóm ngành*

In [ ]:
# Mapping industries into groups
industry_to_sector = {
    'CRYPTO': 'Technology',
    'DATA': 'Technology',
    'HARDWARE': 'Technology',
    'SOFTWARE': 'Technology',
    'HEALTHCARE': 'Healthcare',
    'FITNESS': 'Healthcare',
    'RETAIL': 'Retail',
    'FOOD': 'Retail',
    'FINANCE': 'Finance',
    'EDUCATION': 'Education',
    'MEDIA': 'Media',
    'TRANSPORTATION': 'Transportation and Logistics',
    'LOGISTICS': 'Transportation and Logistics',
    'INFRASTRUCTURE': 'Other',
    'PRODUCT': 'Other',
    'CONSTRUCTION': 'Other',
    'OTHER': 'Other',
    'LEGAL': 'Other',
    'REAL ESTATE': 'Other',
    'HR': 'Other',
    'RECRUITING': 'Other',
    'SALES': 'Sales',
    'AEROSPACE': 'Other',
    'SECURITY': 'Other'
}

### Tình hình sa thải trong các ngành ở các quốc gia

In [ ]:
df_sub=df.copy()
df_sub['sector'] = df_sub['industry'].map(industry_to_sector)

# Group the data by country and industry and calculate the total number of layoffs
# df_sub = df[df['country'] != "UNITED STATES"]
grouped = df_sub.groupby(['country', 'sector'])['total_laid_off'].sum()

# Reshape the data to create a pivot table with countries as rows, industries as columns, and total layoffs as values
pivoted = grouped.reset_index().pivot(index='country', columns='sector', values='total_laid_off')

# Create the stacked bar chart
fig, ax = plt.subplots(figsize=(12, 8))
pivoted.plot(kind='bar', stacked=True, ax=ax)

# Set the y-axis scale to a log scale
ax.set_yscale('log')

# Set the title and axis labels
plt.title('Total Layoffs by Sector and Country')
plt.xlabel('Country')
plt.ylabel('Total Layoffs')

# Show the plot
plt.show()

Nhóm các ngành nghề có liên quan thành các lĩnh vực: **Technology, Healthcare, Retail, Finance, Education, Media, Transportation and Logistics, Other, Sales**.\
Biểu diễn bằng biểu đồ cột chồng cho thấy tỉ lệ các lĩnh vực xuất hiện sa thải ở từng quốc gia, *biểu thị đầy đủ về các lĩnh vực* đang được tác động trong một đất nước như thế nào.\
Các yếu tố đáng chú ý về tỷ lệ sa thải có thể bao gồm:
- Ấn Độ là quốc gia gặp phải vấn đề sa thải nghiêm trọng nhất liên quan đến giáo dục.
- Netherlands chỉ gặp vấn đề ở 2 lĩnh vực chính là Healthcare và Finance.
- Lĩnh vực Finance có nguy cơ bị sa thải cao nhất và bị ảnh hưởng tại hầu hết các quốc gia.

### Các quốc gia đang dẫn đầu trong làn sóng sa thải ở từng lĩnh vực

In [ ]:
# Map industries to sectors in the DataFrame
df['sector'] = df['industry'].map(industry_to_sector)

# Group the data by sector and country
grouped = df.groupby(['sector', 'country'])['total_laid_off'].sum().reset_index()


fig= plt.figure(figsize=(12,10))
# Loop through the sectors and create a bar chart for each one
for index, sector in enumerate(grouped['sector'].unique()):

    # Filter the data for the current sector
    sector_data = grouped[grouped['sector'] == sector]
    
    # Sort the data by total layoffs for each country in the sector
    sorted_data = sector_data.sort_values('total_laid_off', ascending=False)
    
    # Filter to include only the top 5 countries
    top_5 = sorted_data.head(5)
    
    # Create the bar chart
    ax = fig.add_subplot(4,3,index+1)
    ax = sns.barplot(data=top_5,
                     x= "country",
                     y='total_laid_off')
    # ax.bar(top_5['country'], top_5['total_laid_off'])
    ax.set_title(f'Top 5 total Layoffs in {sector}')
    if index % 3 ==0 : ax.set_ylabel('Total LayOffs')
    ax.tick_params(axis='x')


top_n_countries = 10
# create a chart that ranks countries based on their total layoffs in all industries
all_data = df.groupby('country')['total_laid_off'].sum().nlargest(top_n_countries)
ax = fig.add_subplot(4,3,(10,12))
# fig.subplots_adjust(left=0.1, bottom=0.2)
sns.barplot(data=all_data.to_frame().reset_index(),x="country", y ="total_laid_off")
# ax.bar(all_data.index, all_data.values)
ax.set_title(f'Top {top_n_countries} Total Layoffs')
ax.set_xlabel('Country')
ax.set_ylabel('Total Laid Off')
ax.set_yscale('log')

# add padding between subplots and adjust layout
plt.tight_layout(pad=2)

# add margin between rows
fig.subplots_adjust(hspace=0.5)
plt.show()

Từ những nhóm lĩnh vực mà ta đã chia, ta có các biểu đồ cột thể hiện những quốc gia với số lượng sa thải nhân viên ***đạt top đầu***.\
Dễ thấy Mỹ là quốc gia đứng đầu trong tất cả các bảng cho thấy tỉ lệ sa thải ở đất nước này là *đáng báo động*.\
Đặc biệt đáng quan tâm về tỉ lệ bị sa thải của trong các ngành công nghệ liên quan tới Sales, Retail, Technology.

In [ ]:
plt.figure(figsize=(14, 8))
pivot_df = pd.pivot_table(df, values='total_laid_off', columns=['country'], index=['industry'], aggfunc='sum')
# sns.heatmap(pivot_df, cmap='YlGnBu')
import matplotlib.colors as colors
sns.heatmap(pivot_df, cmap='YlGnBu', norm=colors.LogNorm())

**Heat map** trực quan số lượng nhân viên bị cho nghỉ việc ở các quốc gia.\
Mỹ đứng đầu ở hầu hết mọi lĩnh vực.\
Tiếp đến là Ấn Độ với các ngành như: Consumer, Education, Food, Finance, Transportation.

### Ngành nào đang có số lượng layoff cao nhất trên thế giới - Yến 

In [ ]:
industry_grouped = df.groupby(["industry"])["total_laid_off"].sum().reset_index()

plt.figure(figsize=(14, 8))
plt.xticks(rotation=90)
sns.barplot(df,x="industry",y="total_laid_off", errorbar=None)

Ở thời điểm hiện tại xét trên những mảng của các công ty công nghệ, nhận thấy là các công ty chuyên Hardware đang có lượng người bị sa thải nhiều nhất (gấp hơn 2 lần so với công ty đứng vị trí thứ 2).\
Một số nguyên nhân chính của nó là do: sự cạnh tranh, nhu cầu thị trường thay đổi, giảm thiểu chi phí bảo trì và sửa chữa bằng cách tạo ra các sản phẩm dễ dàng sửa chữa, tác động của đại dịch COVID-19.

### Xét tất cả các ngành, dùng biểu đồ cột thể hiện số lượng quốc gia có ngành nghề đó layoff top đầu / cuối. Hưng 

Các bước thực hiện: Với mỗi industry ta sẽ tìm số lượng country laid off:
- Lọc ra các country
- Với mỗi country, đếm số lượng total_laid_off
- Nếu total_laid_off của country tương ứng khác 0 thì số lượng country laid off tăng lên 1
Sau khi đã có số lượng country laid off, ta trực quan bằng biểu đồ cột với trục x là industry và y là số lượng country laid off tương ứng với industry đó. (xem hình bên dưới)

Nhận xét:
- Industry có số lượng country laid off cao nhất là finance (số lượng: 20)
- Industry có số lượng country laid off thấp nhất là manufacturing (số lượng: 1)

In [ ]:
x= df["industry"].unique()
print(len(x))
y=[]
z=0

for i in range(len(df["industry"].unique())):
    industryData= df["industry"].unique()[i]
    industryRows = df[df["industry"]==industryData]
    countryArr = industryRows["country"].unique()

    res1=0
    for item in industryRows["total_laid_off"]:
        res1+=item
    # print(res1)


    totalArrCountry = []

    for item in countryArr:
        rows = industryRows[industryRows["country"]==item]
        totalArr = rows["total_laid_off"]
        sumTmp=0
        for total in totalArr:
            sumTmp+=total
        # print(sumTmp)
        totalArrCountry.append(sumTmp)

    result=0
    for item in range(len(countryArr)):
        if totalArrCountry[item]!=0:
            result+=1

    y.append(result)
    z+=result


d = {'industry': x, 'total countries': y}
totalCountryByIndustry = pd.DataFrame(data=d)
# totalCountryByIndustry

print(totalCountryByIndustry[totalCountryByIndustry["total countries"]==totalCountryByIndustry["total countries"].max()])
print(totalCountryByIndustry[totalCountryByIndustry["total countries"]==totalCountryByIndustry["total countries"].min()])



sns.set(font_scale=0.6)
# sns.set(rc = {'figure.figsize':(25,8)})
plt.xticks(rotation=90)
sns.barplot(data=totalCountryByIndustry, x="industry", y="total countries",)




### Thêm total_employee để chọn ra top company theo quy mô nhân sự. Từ đây phát triễn các phân tích trên nhóm top company. Yến  

In [ ]:
n=10

top_n_biggest_companies =list(df_with_total_employees.groupby("company")\
                                                .agg({"total_employees": "max"})\
                                                .sort_values("total_employees",ascending=False)[:n]\
                                                .index)


plt.figure(figsize=(14, 8))
plt.xticks(size=10)
plt.title("Tổng nhân viên trong các tập đoàn lớn",size=14)
sns.barplot(df_with_total_employees[df_with_total_employees['company'].isin(top_n_biggest_companies)],
            x="company",
            y="total_employees",
            errorbar=None,
            estimator="sum")


plt.figure(figsize=(14, 8))
plt.xticks(size=10)
plt.title("Tổng nhân sự sa thải trong các tập đoàn lớn",size=14)
sns.barplot(df_with_total_employees[df_with_total_employees['company'].isin(top_n_biggest_companies)],
            x="company",
            y="total_laid_off",
            errorbar=None,
            estimator="sum")

In [ ]:
df_with_total_employees.describe()['total_laid_off']

Trong top 10 công ty lớn, số lượng sa thải đạt thấp nhất là khoảng 2200 người. Nhận thấy được là các tập đoàn lớn có xu hướng sa thải nhân viên cao hơn mức bình quân của toàn bộ các công ty trong tập dữ liệu (272 người).

### Câu 16
Có 1 số điểm thú vị trong dữ liệu đang tìm từ từ. VD: 

 1 số company mà percentage_laid_off=1 tức là toàn bộ công ty đó bị sa thải --> bankrupt. -> KHÁM PHÁ PHÁ SẢN 

Công ty nào sa thải nhân viên và những dịp lễ đặc biệt (eg: Christmas...) nhiều nhất? Số lượng? 

Thời điểm nào lay off nhiều nhất (Tháng/quý/năm - stack bar) 

### Địa điểm tiêu biểu nơi xảy ra lay off

In [ ]:

plt.figure(facecolor='k')
wordcloud = WordCloud(width=400, height=200,max_font_size=100).generate(' '.join(df['location']))

plt.imshow(wordcloud,interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

San Francisco area bay với Silicon Valley nổi tiếng lại cũng là nơi diễn ra lay off nhiều nhất :(

In [ ]:
df_clean_total_lo=df[df['total_laid_off']>0]
df_clean_total_lo['date'].min()
df_clean_total_lo['date'].max()
df_clean_total_lo['month_year'] = pd.to_datetime(df_clean_total_lo['date']).dt.strftime('%Y-%m')
total_by_month_year = df_clean_total_lo.groupby('month_year').sum()['total_laid_off']
total_by_month_year.reindex(pd.period_range('2020-01',total_by_month_year.index[-1],freq='M'))
total_by_month_year.fillna(0.0)
total_by_month_year.max()
# lo_2020=lo_2020.reindex(pd.RangeIndex(1,13),fill_value= 0).ffill()
# total_by_month_year.reset_index().rename(columns={'sum':'valuesum'})



## Số lượng lay off theo thời gian

In [ ]:
df_total_employees_time=df_with_total_employees[df_with_total_employees.total_laid_off>0]
df_total_employees_time['year']=df_total_employees_time['date'].apply(lambda x :x.year)
df_total_employees_time['month']=df_total_employees_time['date'].apply(lambda x :x.month)
df_total_employees_time['month_year'] = pd.to_datetime(df_total_employees_time['date']).dt.strftime('%Y-%m')



In [ ]:
total_by_time=df_total_employees_time.groupby('month_year').sum()['total_laid_off']

fig, ax = plt.subplots( figsize=(9, 5), tight_layout=True)
fig.tight_layout(pad=5.0)
plt.plot(total_by_time.index,total_by_time.values)
plt.xlabel('Date')
plt.ylabel('Total laid off')
plt.title('Số người bị sa thải qua từng thời điểm')


x_max=np.where(total_by_time == total_by_time.max())[0]
y_max=total_by_time.max()
plt.annotate(f'{int(y_max)} employees laid off', xy=(x_max, y_max), xytext=(x_max-10, y_max),arrowprops=dict(arrowstyle='->',color='red'))
# beautify the x-labels
plt.gcf().autofmt_xdate()
print(f"Số lượng lay off nhiều nhất trong 1 tháng: {y_max} người")

Làng sóng lay off diễn ra mạnh mẽ nhất từ tháng 9 2022 và đạt đỉnh vào tháng 1 2023 với tổng cộng gần 80 ngàn người bị sa thải trong 1 tháng.

### Số lượng lay off theo tháng

In [ ]:

analysis_year = df_total_employees_time.groupby(["year"],sort=True)
fig, ax = plt.subplots( figsize=(9, 5), tight_layout=True)
fig.tight_layout(pad=5.0)
for year, dfx in analysis_year:
    df_m=dfx.groupby('month').sum()['total_laid_off']
    plt.plot(df_m.index,df_m.values,label=f'{year}')

plt.xticks(np.arange(1, 13, 1))
plt.xlabel('Month')    
plt.ylabel('Total laid off')   
plt.title('Số người bị sa thải theo tháng') 
plt.legend()


Tháng 11 và tháng 1 là hai thời điểm có nhiều người bị sa thải nhất. Nhìn rộng hơn thì quý 1 và quý 4 là những thời điểm có lượng sa thải lớn nhất.

## Liệu có đợt lay off này diễn ra vào holiday week?

*holiday week: Tuần lễ cuối cùng của năm gồm cả Christmas (25/12) và ngày đầu năm mới (01/01). Đây là dịp lễ lớn ở các nước phương tây.

In [ ]:
df_time=df.set_index('date',drop=True)
holiday_laid_off_2021=df_time['2020-12-24':'2021-01-01']
holiday_laid_off_2022=df_time['2021-12-24':'2022-01-01']
holiday_laid_off_2023=df_time['2022-12-24':'2023-01-01']

In [ ]:
print(holiday_laid_off_2021)

In [ ]:
print(holiday_laid_off_2022)

In [ ]:
print(holiday_laid_off_2023)


Kết quả ta thấy cuối năm 2022 có 4 công ty cho một số lượng khá lớn nhân viên nghỉ khỏe ngay trong thời gian diễn ra trong holiday week 😥.

### Với mỗi đợt sa thải, các công ty thường cho nghỉ bao nhiêu phần trăm nhân sự?

In [ ]:
df_clean_percentage=df[df['percentage_laid_off']>0]
df_clean_percentage['percentage_laid_off']=df_clean_percentage['percentage_laid_off']*100
df_clean_percentage['percentage_lo_range']=pd.cut(x=df_clean_percentage['percentage_laid_off'], bins=[0, 10, 20, 30,
                                          40,50,60,70,80,90,100],labels=['0-10', '10-20', '20-30',
                                          '30-40','40-50','50-60','60-70','70-80','80-90','90-100'],include_lowest=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), tight_layout=True)
fig.tight_layout(pad=5.0)
ax.set_xticks(np.arange(0, 101, 10))
ax.set_title( 'Số đợt sa thải theo quy mô phần trăm nhân sự mỗi đợt',wrap=True,pad=10)
ax.set_xlabel('percentage laid off (%)')
ax.set_ylabel('Number of times laid off happend')
sns.histplot(df_clean_percentage['percentage_laid_off'], kde=True, bins=50)

Ta thấy biểu đồ có 2 đỉnh. Đỉnh lớn nằm quanh 10% cho ta nhận định với mỗi đợt sa thải thì khoảng 10% nhân sự là con số ưu thích của các công ty. Biểu đồ lệch trái cho thấy các công ty có xu hướng luôn có gắng để sa thải ít hơn 50% nhân sự mỗi đợt. Điều thú vị khác là đỉnh thứ 2 nằm ở mức 100%. Điều này có thể liên quan đến các công ty quy nhỏ, toàn bộ các thành viên gắng kết với nhau đến những giây phút cuối cùng ?!?

### Top 20 công ty có tổng % số lượng nhân sự bị sa thải nhiều nhất

In [ ]:
fig, ax = plt.subplots( figsize=(9, 5), tight_layout=True)
fig.tight_layout(pad=5.0)
top_20 = df.groupby(['company','location']).sum().sort_values(by='percentage_laid_off',ascending=False).head(20)['percentage_laid_off'] * 100.0
top_20.plot(kind='barh',ax=ax)
ax.set_title('Top 20 công ty sai thải nhân sự nhiều nhất theo tổng tỉ lệ phần trăm')
ax.set_xlabel('percentage laid off (%)')
ax.set_ylabel('Company')

Ta có thể thấy là có nhiều công ty sa thải với tổng hơn cả 100% nhân sự. Nguyên do là vì họ đã sa thải- tuyển lại và tiếp tục sa thải tiếp trong nhiều đợt . Một ví dụ là công ty `Air rooms`, ngày 2020-04-22 họ sa thải 70% nhân sự và sau đó vào ngày 2020-05-07 họ tiếp tục sa thải toàn bộ 100% nhân sự. Cá biệt có công ty `Service` trụ sở tại Los Angeles đã sai thải 2 lần với quy mô mỗi lần là 100% nhân sự chỉ cách nhau vài ngày 💀.

In [ ]:
print(df[df.company=='AIRY ROOMS'])
print(df[df.company=='SERVICE'])

### Tìm hiểu về các công ty sa thải hơn 50% nhân sự trong các đợt

In [ ]:
more_than_50percent_laid_off=df_clean_percentage[df_clean_percentage.percentage_laid_off>=50]
# số lần sa thải có quy mô > 50% nhân viên
n_more_than_50percent_laid_off=more_than_50percent_laid_off.shape[0]

# số lần sa thải có quy mô 100% nhân viên
laid_off_all_employees =df_clean_percentage[df_clean_percentage.percentage_laid_off==100]
n_laid_off_all_employees = laid_off_all_employees.shape[0]
print("Tổng số đợt sa thải >50% nhân sự: "+str(n_more_than_50percent_laid_off))
print("Tổng số đợt sa thải 100% nhân sự: "+str(n_laid_off_all_employees))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), tight_layout=True)
fig.tight_layout(pad=5.0)
#pie chart
pie_lb=['\n'.join(wrap('Vẫn còn nhân sự', 15)),'\n'.join(wrap('Toàn bộ nhân sự đã bị sa thải', 15))]
ax1.pie([n_more_than_50percent_laid_off-n_laid_off_all_employees,n_laid_off_all_employees],labels=pie_lb,autopct='%.0f%%',counterclock=False, startangle=-270,colors=['#F1C40F','#E74C3C'])
ax1.set_title('Tình trạng nhân sự của các công ty có hơn 50% nhân viên bị sa thải',wrap=True)

#histogram
ax2.hist(more_than_50percent_laid_off['stage'],alpha=0.9)
plt.xticks(rotation=45, size=10)
ax2.set_xlabel('Stage')
ax2.set_ylabel('Number of companies')
ax2.set_title('Phân bố theo Stage của các công ty có hơn 50% nhân viên bị sa thải',wrap=True)
plt.show()

Dựa vào biểu đồ tròn ta thấy rằng, một khi đã sa thải hơn 50% nhân sự thì có tới hơn 53% đó sẽ là những lần sa thải toàn bộ 100% nhân sự. 
Nhìn vào biểu đồ histogram ta thấy rằng đa số những công ty thuộc nhóm ra thải nhiều này chủ yếu không có thông tin về `stage` hoặc thuộc những giai đoạn khởi đầu (`Seed`, `SeriesA`...). Những công ty này thiếu sự ổn định.

### Tìm hiểu quan hệ giữa total_laid_off,percentage_laid_off,funds_raised,total_employees

In [ ]:
# xữ lý các dòng có giá trị 0
df_cor=df_with_total_employees[(df_with_total_employees != 0).all(1)]
df_cor.reset_index(drop=True)
# lấy 4 cột 'total_laid_off','percentage_laid_off','funds_raised','total_employees' để so sánh quan hệ
df_cor=df_cor.loc[:,['total_laid_off','percentage_laid_off','funds_raised','total_employees']]
df_cor.shape[0]

In [ ]:
# vẽ correlation map
corr = df_cor.corr()
sns.heatmap(corr,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True)

Ta thấy rằng `total_employees` và `total_laid_off` có quan hệ đồng biến rất mạnh còn các cặp feature còn lại gần như không có quan hệ.

In [ ]:
sns.scatterplot(data=df_cor,x='total_laid_off',y='total_employees')